In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 25.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 17.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317129 sha256=183306d1d6219a631581d410ddca97cde8e937585b61ca3c57a143f953daca36
  Stored in directory: /Users/dayookubanjo/Library/Caches/pip/wheels/fc/49/ad/5c21e362b2cc9fb6785cdf03f7864b96d8ca6521f1947e3e25
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('stocks_etf_ai').getOrCreate()

23/04/29 07:41:18 WARN Utils: Your hostname, Adedayos-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.2.27 instead (on interface en0)
23/04/29 07:41:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/29 07:41:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [118]:
from pyspark.sql.functions import  input_file_name
symbols_file_path = './stocks_etf_ai_airflow/data/symbols_valid_meta.csv'
symbols_df = spark.read.csv(symbols_file_path, header=True, inferSchema=True)

In [119]:
symbols_df.printSchema()

root
 |-- Nasdaq Traded: string (nullable = true)
 |-- Symbol: string (nullable = true)
 |-- Security Name: string (nullable = true)
 |-- Listing Exchange: string (nullable = true)
 |-- Market Category: string (nullable = true)
 |-- ETF: string (nullable = true)
 |-- Round Lot Size: double (nullable = true)
 |-- Test Issue: string (nullable = true)
 |-- Financial Status: string (nullable = true)
 |-- CQS Symbol: string (nullable = true)
 |-- NASDAQ Symbol: string (nullable = true)
 |-- NextShares: string (nullable = true)



In [126]:
from pyspark.sql import functions as F
from pyspark.sql.functions import  input_file_name
main_file_path = './stocks_etf_ai_airflow/data/**/*.csv'
main_df = spark.read.csv(main_file_path, header=True, inferSchema=True)
main_df = main_df.withColumn("filename",  F.element_at(F.split(F.input_file_name(), "/"),-1)   )
main_df = main_df.withColumn('name', split(main_df['filename'], '\.').getItem(0))
main_df.select('name').show(3)

+----+
|name|
+----+
| HPQ|
| HPQ|
| HPQ|
+----+
only showing top 3 rows



In [127]:
main_df = main_df.drop('filename')
main_df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- name: string (nullable = true)



In [128]:
from pyspark.sql.functions import *

new_df = main_df.join(symbols_df, main_df.name == symbols_df.Symbol).select(
    symbols_df['Symbol'], symbols_df['Security Name'], 
    main_df['*'])

In [129]:
new_df.show(3)

+------+--------------------+----------+-------------------+-------------------+-------------------+-------------------+--------------------+---------+----+
|Symbol|       Security Name|      Date|               Open|               High|                Low|              Close|           Adj Close|   Volume|name|
+------+--------------------+----------+-------------------+-------------------+-------------------+-------------------+--------------------+---------+----+
|   HPQ|HP Inc. Common Stock|1962-01-02|0.13127270340919495|0.13127270340919495|0.12417688220739365|0.12417688220739365| 0.00688728503882885|2480300.0| HPQ|
|   HPQ|HP Inc. Common Stock|1962-01-03|0.12417688220739365|0.12417688220739365|0.12151595205068588|0.12284641712903976|0.006813489831984043| 507300.0| HPQ|
|   HPQ|HP Inc. Common Stock|1962-01-04|0.12284641712903976|  0.126837819814682|0.11796803772449493|  0.120185486972332|0.006665901280939579| 845500.0| HPQ|
+------+--------------------+----------+------------------

In [130]:
new_df = new_df.drop('name')
new_df.show(3)

+------+--------------------+----------+-------------------+-------------------+-------------------+-------------------+--------------------+---------+
|Symbol|       Security Name|      Date|               Open|               High|                Low|              Close|           Adj Close|   Volume|
+------+--------------------+----------+-------------------+-------------------+-------------------+-------------------+--------------------+---------+
|   HPQ|HP Inc. Common Stock|1962-01-02|0.13127270340919495|0.13127270340919495|0.12417688220739365|0.12417688220739365| 0.00688728503882885|2480300.0|
|   HPQ|HP Inc. Common Stock|1962-01-03|0.12417688220739365|0.12417688220739365|0.12151595205068588|0.12284641712903976|0.006813489831984043| 507300.0|
|   HPQ|HP Inc. Common Stock|1962-01-04|0.12284641712903976|  0.126837819814682|0.11796803772449493|  0.120185486972332|0.006665901280939579| 845500.0|
+------+--------------------+----------+-------------------+-------------------+--------

In [131]:
new_df.write.mode('overwrite').parquet("transformed_stocks_etf_data.parquet")

23/04/29 09:51:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/04/29 09:51:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/04/29 09:51:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/04/29 09:51:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/04/29 09:51:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 61.43% for 11 writers
23/04/29 09:51:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 56.31% for 12 writers
23/04/29 09:51:50 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,

23/04/29 09:52:05 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/04/29 09:52:05 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/04/29 09:52:05 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/04/29 09:52:05 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/04/29 09:52:05 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 61.43% for 11 writers
23/04/29 09:52:05 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 56.31% for 12 writers
23/04/29 09:52:08 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,

23/04/29 09:52:17 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/04/29 09:52:17 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/04/29 09:52:17 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/04/29 09:52:17 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/04/29 09:52:17 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 61.43% for 11 writers
23/04/29 09:52:17 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 56.31% for 12 writers
23/04/29 09:52:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,

23/04/29 09:52:25 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 61.43% for 11 writers
23/04/29 09:52:25 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/04/29 09:52:25 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/04/29 09:52:25 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/04/29 09:52:25 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/04/29 09:52:26 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/04/29 09:52:26 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,0

23/04/29 09:52:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 61.43% for 11 writers
23/04/29 09:52:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/04/29 09:52:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/04/29 09:52:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/04/29 09:52:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/04/29 09:52:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/04/29 09:52:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,0